In [73]:

import tensorflow as tf
import random as rn
import os
from collections import Counter


os.environ['PYTHONHASHSEED']='0'

#FONTE: https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/

#ATTENTO: SE MI SALVO IL MODELLO NON HO BISOGNO DI DATI RIPRODUCIBILI E POSSO USARE IL PARALLELISMO
#session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
#                              inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)
import os



os.environ['PYTHONHASHSEED']='0'

#FONTE: https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/

#ATTENTO: SE MI SALVO IL MODELLO NON HO BISOGNO DI DATI RIPRODUCIBILI E POSSO USARE IL PARALLELISMO
#session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
#                              inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)



import pandas as pd
import numpy as np  
from sklearn.model_selection import train_test_split   
from utility import Utility
from dataset import Dataset
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from collections import Counter
from keras.layers import Dense, Conv2D, Flatten
from keras.activations import relu, elu, sigmoid,softmax
from keras.losses import categorical_crossentropy, logcosh,binary_crossentropy
from talos.metrics.keras_metrics import  precision_acc, recall_acc, fmeasure_acc #https://github.com/netrack/keras-metrics/blob/master/keras_metrics/metrics.py
from talos.model.normalizers import lr_normalizer
from talos import live
import talos as ta
from keras.optimizers import Adam, Nadam, RMSprop
from sklearn.metrics import accuracy_score

X_train=pd.read_csv("X_train.csv", delimiter=",").values
X_val=pd.read_csv("X_val.csv", delimiter=",").values
y_train=pd.read_csv("y_train.csv", delimiter=",").values
y_val=pd.read_csv("y_val.csv", delimiter=",").values

In [74]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((413, 14), (137, 14), (413, 1), (137, 1))

In [75]:
def getCNNParams():
        from keras.optimizers import Adam, Nadam, RMSprop

        p= {'lr': [4.0002],
            'first_neuron':[8],
            'hidden_layers': [2],
            'batch_size': [4],
            'epochs': [100],
            'dropout': [0.12],
            'weight_regulizer': [None],
            'emb_output_dims': [None],
            'optimizer': [Adam],
            'losses': [binary_crossentropy],
            'activation': [elu],
            'last_activation': [sigmoid]}
        
        """
        p= {'lr': (1, 5, 10),
           'first_neuron':[8,16,32],
           'hidden_layers': [1,2, 3, 4,5,6],
           'batch_size': [4,8,16,32,64],
           'epochs': [500,800,1000],
           'dropout': (0, 0.3, 10),
           'weight_regulizer': [None],
           'emb_output_dims': [None],
           'optimizer': [Adam, RMSprop],
           'losses': [binary_crossentropy],
           'activation': [relu,elu],
           'last_activation': [sigmoid]}"""
        return p 
    
def getCNNModel( x_train, y_train, x_val, y_val, params):
        
        model = Sequential()
        model.add(Dense(params['first_neuron'],
                        input_dim=x_train.shape[1],
                        activation=params['activation']))
        #hidden layer
        ta.model.hidden_layers(model, params, 1)
        
        model.add(Dense(1, activation=params['last_activation']))
    
        model.compile(optimizer=params['optimizer'](lr_normalizer(params['lr'], params['optimizer'])),
                      loss=params['losses'],
                      metrics=['acc',fmeasure_acc,precision_acc, recall_acc])
    
       
        
        history = model.fit(x_train, y_train,
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        verbose=0,
                        validation_data=[x_val, y_val])
                        #callbacks=[live()])  da aggiungere se si vuole vedere il plotting durante le epoche ma molto leto
    
        return history, model

params=getCNNParams()

scan = ta.Scan(X_train, y_train,
        params=params,
        dataset_name="name",
        x_val=X_val,
        y_val=y_val,
        experiment_no="1",
        model=getCNNModel,
        grid_downsample=1)

100%|██████████| 1/1 [00:08<00:00,  8.61s/it]

Scan Finished!


In [76]:
ta.Deploy(scan,"experiment",metric= "val_acc")

Deploy package experiment have been saved.


In [77]:
scan.data

,round_epochs,val_loss,val_acc,val_fmeasure_acc,val_precision_acc,val_recall_acc,loss,acc,fmeasure_acc,precision_acc,...,hidden_layers,batch_size,epochs,dropout,weight_regulizer,emb_output_dims,optimizer,losses,activation,last_activation
0,100,0.43787189115545827,0.8248175182481752,0.6257212035847406,0.6715328162604005,0.6788320967750828,0.4101038930202512,0.8353510895883777,0.7479072444086791,0.7627118378516837,...,2,4,100,0.12,None,None,<class 'keras.optimizers.Adam'>,<function binary_crossentropy at 0x7fc719cb82f0>,<function elu at 0x7fc719cdcd90>,<function sigmoid at 0x7fc719c6b158>


In [78]:
restore=ta.Restore("experiment.zip")

In [79]:
model=restore.model
y_pred_train=model.predict_classes(X_train)
(y_pred_train==y_train).mean()

0.8087167070217918

In [80]:
model.compile(optimizer=Adam(lr=4.0002), loss='binary_crossentropy', metrics=['acc'] )
loss, metric=model.evaluate(X_train, y_train)
loss, metric

413/413 [==============================] - 0s 116us/step


(0.42492127432950183, 0.8087167075990764)

In [81]:
accuracy_score(y_pred_train, y_train)   #here seems to work 

0.8087167070217918

In [86]:
#NOW IF I TRY TO RESTORE PREVIOUS RESULTS WITH A LARGE SCAN SOMETHING DIDN'T WORK
r_acc4=ta.Restore('australian23-01-2019 19:42:36val_acc.zip')
model4=r_acc4.model
model4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 120       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dropout_1 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 72        
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 9         
Total params: 273
Trainable params: 273
Non-trainable params: 0
_________________________________________________________________


In [89]:
r_acc4.results.sort_values("val_acc", ascending=False).head() 

,round_epochs,val_loss,acc,loss,val_precision_acc,val_acc,val_recall_acc,fmeasure_acc,recall_acc,precision_acc,...,batch_size,activation,emb_output_dims,last_activation,first_neuron,hidden_layers,optimizer,lr,losses,epochs
268,1000,0.577316,0.927536,0.294325,0.816425,0.920290,0.809179,0.844904,0.888889,0.859098,...,2,<function elu at 0x7f5dbcc10a60>,None,<function sigmoid at 0x7f5dbcc10d90>,8,4,<class 'keras.optimizers.Adam'>,4.0002,<function binary_crossentropy at 0x7f5dbcc60bf8>,1000
798,1000,0.679485,0.944444,1.236957,0.915942,0.913043,0.899517,0.925427,0.968760,0.951691,...,5,<function relu at 0x7f5dbcc10c80>,None,<function sigmoid at 0x7f5dbcc10d90>,8,8,<class 'keras.optimizers.RMSprop'>,2.5005,<function binary_crossentropy at 0x7f5dbcc60bf8>,1000
562,800,0.919119,0.963768,0.162795,0.934299,0.913043,0.873752,0.959356,0.969973,0.966484,...,4,<function elu at 0x7f5dbcc10a60>,None,<function sigmoid at 0x7f5dbcc10d90>,32,32,<class 'keras.optimizers.Adam'>,4.0002,<function binary_crossentropy at 0x7f5dbcc60bf8>,800
1007,500,0.639003,0.905797,0.841443,0.782609,0.913043,0.743961,0.793927,0.802737,0.842995,...,5,<function relu at 0x7f5dbcc10c80>,None,<function sigmoid at 0x7f5dbcc10d90>,16,4,<class 'keras.optimizers.RMSprop'>,3.0004,<function binary_crossentropy at 0x7f5dbcc60bf8>,500
916,500,0.435252,0.893720,0.431952,0.915942,0.913043,0.894686,0.866625,0.877295,0.925374,...,1,<function relu at 0x7f5dbcc10c80>,None,<function sigmoid at 0x7f5dbcc10d90>,8,8,<class 'keras.optimizers.RMSprop'>,3.0004,<function binary_crossentropy at 0x7f5dbcc60bf8>,500


In [90]:
model4=r_acc4.model
y_pred_train=model4.predict_classes(X_train)
(y_pred_train==y_train).mean()


0.9007263922518159

In [91]:
accuracy_score(y_pred_train, y_train)  #this should be 92

0.9007263922518159